In [1]:
def calculate_metrics(true_edges, predicted_edges):
    """
    计算精准率、召回率，并找出真实中存在但预测中不存在的边，以及预测中存在但真实中不存在的边。
    """
    # 真正例（TP）：真实和预测都存在的边
    tp = true_edges & predicted_edges

    # 真实中存在，但预测中不存在的边（FN）
    fn = true_edges - predicted_edges

    # 预测中存在，但真实中不存在的边（FP）
    fp = predicted_edges - true_edges

    # 计算精准率和召回率
    # print("tp={}".format(len(tp)))
    # print("predicted_edges={}".format(len(predicted_edges)))
    # print("true_edges={}".format(len(true_edges)))
    precision = len(tp) / len(predicted_edges) if predicted_edges else 0
    recall = len(tp) / len(true_edges) if true_edges else 0

    # 计算F1分数
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    return precision, recall, f1, fn, fp


import json
from collections import defaultdict
def merge_list_values(pairs):
    """合并重复键的列表值"""
    merged = defaultdict(list)
    for key, value in pairs:
        # 确保值都是列表（如果不是，转换为列表）
        if not isinstance(value, list):
            value = [value]
        merged[key].extend(value)  # 合并列表内容
    
    return dict(merged)
def load_json(name, file_path, replace_list=[], skip_list=[], pre_clean=[],all_or_EA = 1):
    """
    加载 JSON 文件并将边关系转换为集合形式，便于比较。
    返回边集合，格式为 {(source, target), ...}
    """
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)



    edges = set()
    for source, targets in data.items():
        if source.split('.')[0] in skip_list:
            continue
        # if source.split('.')[0] in pre_clean:
        #     source = '.'.join(source.split('.')[1:])
        # if 'lambda' in source:
        #     continue
        for r in replace_list:
            source = source.replace(r[0],r[1])
        # if source in replace_map:
        #     source = replace_map[source]
        # if not source.startswith('<') and not source.startswith('fabric') and not source.startswith('integration'):
        #         continue
        if all_or_EA == 0:
            if not source.startswith('<') and not source.startswith(name):
                continue
        #source = source.split('.')[0] + source.split('.')[-1]
        for target in targets:
            # if target.startswith('<'):
            #     continue
            if "error" in target.lower():
                continue
            # if 'lambda' in target:
            #     continue
            if target.split('.')[0] in skip_list:
                continue
            # if target.split('.')[0] in pre_clean:
            #     target = '.'.join(target.split('.')[1:])
            for r in replace_list:
                target = target.replace(r[0],r[1])

            # if target in replace_map:
            #     target = replace_map[target] 
            # if not target.startswith('<') and not target.startswith('fabric') and not target.startswith('integration'):
            #     continue   
            if all_or_EA == 0:
                if not target.startswith('<') and not target.startswith(name):
                    continue   
            #target = target.split('.')[0] + '.' + target.split('.')[-1]
            edges.add((source, target))

    return edges

def main(name, replace_list, skip_list, pre_clean=[], precision_write = 0, recall_write = 0, Ae_or_pycg = 1, all_or_EA = 1):
    """
    主函数，加载 JSON 文件，计算指标并输出结果。
    """
    true_json_path = "../ground-truth-cgs/{}.json".format(name)  # 真实边的 JSON 文件路径
    #true_json_path = "../PyCG_data/{}.json".format(name)
    if Ae_or_pycg == 1:
        print("{} Ae:".format(name))
        predicted_json_path = "../Ae_data/{}.json".format(name)  # 预测边的 JSON 文件路径
        # if all_or_EA == 0:
        #     predicted_json_path = "../file2class_data/{}.json".format(name)
        # else:
        #     predicted_json_path = "../stdlib_and_thirdlib_data/{}.json".format(name)
    elif Ae_or_pycg == 0 :
        print("{} PyCG:".format(name))
        predicted_json_path = "../PyCG_data/{}.json".format(name)  # 预测边的 JSON 文件路径
    elif Ae_or_pycg == 2 :
        print("{} Depends:".format(name))
        predicted_json_path = "../Depends_data/{}.json".format(name)  # 预测边的 JSON 文件路径
    # try:
    #     import_info_path = "../STAR/pre_knowledge/{}_import_info.json".format(name)
    #     pre_annotations_path = "../STAR/pre_knowledge/{}_pre_annotations.json".format(name)

    #     with open(pre_annotations_path, 'r', encoding='utf-8') as f:
    #         func_info_dict = json.load(f)

    #     for func_name, func_info in func_info_dict.items():
    #         if func_info_dict[func_name]["namespace"] == func_name.split('.')[-1]:
    #             if func_name.split('.')[0] in pre_clean:
    #                 replace_map ['.'.join(func_name.split('.')[1:]) + ".__init__"] = '.'.join(func_name.split('.')[1:])
    #             else :
    #                 replace_map[func_name + ".__init__"] = func_name
    # except:
    #     pass

    

    # 加载边集合
    true_edges = load_json(name, true_json_path, replace_list,skip_list, pre_clean, all_or_EA)
    predicted_edges = load_json(name, predicted_json_path, replace_list, skip_list, pre_clean, all_or_EA)
    # if Ae_or_pycg == 1 and all_or_EA == 0:
    #     predicted_edges = predicted_edges.union(load_json(name, "../stdlib_and_thirdlib_data/{}.json".format(name), replace_list, skip_list, pre_clean, all_or_EA))
    # 计算指标
    precision, recall, f1, fn, fp = calculate_metrics(true_edges, predicted_edges)
    precision = precision*100
    recall = recall*100
    f1 = f1*100
    # 输出结果
    
    print(f"Precision (精准率): {precision:.3f}")
    print(f"Recall (召回率): {recall:.3f}")
    print(f"F1: {f1:.3f}\n\n")

    if recall_write:
        #召回率
        print("Edges in true JSON but not in predicted JSON (FN):")
        for edge in sorted(fn):
            print(edge)

    if precision_write:
        # 精准率
        print("\nEdges in predicted JSON but not in true JSON (FP):")
        for edge in sorted(fp):
            print(edge)
    return precision, recall, f1

In [8]:
project_name_list_1 = ["asciinema","autojump","fabric","face_classification","Sublist3r"]
project_name_list_2 = ['bpytop','furl','rich_cli','sqlparse','sshtunnel','textrank4zh']
project_name_list_1 = ["asciinema","autojump"] #测试集
project_name_list_2 = ['sqlparse','sshtunnel','textrank4zh'] #测试集
pre_clean = []
#skip_list = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>"),
    ('Socket','socket'),
    ('<builtin>','<builtin>'),
    ('Socket','socket'),
    ('invoke.config.Config._set','invoke.config.DataProxy._set'),
    ('invoke.Context._set','invoke.config.DataProxy._set'),
    ('fabric.util.debug','util.debug'),
    ('invoke.terminals.pty_size','invoke.pty_size'),
    ('invoke.context.Context._run',"invoke.Context._run"),
    ('invoke.context.Context._sudo',"invoke.Context._sudo"),
    ('invoke.parser.argument.Argument',"invoke.Argument"),
    ('invoke.program.Program.core_args',"invoke.Program.core_args"),
    ('invoke.collection.Collection',"invoke.Collection"),
    ('invoke.program.Program.load_collection',"invoke.Program.load_collection"),
    ('invoke.program.Program.no_tasks_given',"invoke.Program.no_tasks_given"),
    ('invoke.program.Program.update_config',"invoke.Program.update_config"),
    ('invoke.collection.Collection',"invoke.Collection"),
    ('re.compile.findall','re.findall'),
    ('requests.sessions.Session.get','requests.Session.get'),
    ('argparse.ArgumentParser.add_argument','argparse.add_argument'),
    ('argparse.ArgumentParser.parse_args','argparse.parse_args'),
    ('threading.Thread.start','threading.start')
]

# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])

precision_all_Ae = 0
recall_all_Ae = 0
f1_all_Ae = 0

precision_all_pycg = 0
recall_all_pycg = 0
f1_all_pycg = 0

precision_all_depends = 0
recall_all_depends= 0
f1_all_depends = 0
for name in project_name_list_1:
    if name == "autojump":
        replace_list.append(('bin.',''))
    elif name == "face_classification":
        replace_list.append(('src.',''))
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=1, all_or_EA=1)
    precision_all_Ae += precision
    recall_all_Ae += recall
    f1_all_Ae += f1
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
    precision_all_pycg += precision
    recall_all_pycg += recall
    f1_all_pycg += f1
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)
    precision_all_depends += precision
    recall_all_depends += recall
    f1_all_depends += f1
    if name == "autojump":
        replace_list.pop()
    elif name == "face_classification":
        replace_list.pop()
    
#project_name_list_2 = []
for name in project_name_list_2:
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
    precision_all_Ae += precision
    recall_all_Ae += recall
    f1_all_Ae += f1
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
    precision_all_pycg += precision
    recall_all_pycg += recall
    f1_all_pycg += f1
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)
    precision_all_depends += precision
    recall_all_depends += recall
    f1_all_depends += f1

print("Ae average pre={:.3f}".format(precision_all_Ae/len(project_name_list_1+project_name_list_2)))
print("Ae average rec={:.3f}".format(recall_all_Ae/len(project_name_list_1+project_name_list_2)))
print("Ae average F1={:.3f}".format(f1_all_Ae/len(project_name_list_1+project_name_list_2)))

print("pycg average pre={:.3f}".format(precision_all_pycg/len(project_name_list_1+project_name_list_2)))
print("pycg average rec={:.3f}".format(recall_all_pycg/len(project_name_list_1+project_name_list_2)))
print("pycg average F1={:.3f}".format(f1_all_pycg/len(project_name_list_1+project_name_list_2)))

print("depends average pre={:.3f}".format(precision_all_depends/len(project_name_list_1+project_name_list_2)))
print("depends average rec={:.3f}".format(recall_all_depends/len(project_name_list_1+project_name_list_2)))
print("depends average F1={:.3f}".format(f1_all_depends/len(project_name_list_1+project_name_list_2)))

asciinema Ae:
Precision (精准率): 89.873
Recall (召回率): 81.844
F1: 85.671


asciinema PyCG:
Precision (精准率): 97.845
Recall (召回率): 65.418
F1: 78.411


asciinema Depends:
Precision (精准率): 100.000
Recall (召回率): 18.156
F1: 30.732


autojump Ae:
Precision (精准率): 89.912
Recall (召回率): 78.998
F1: 84.103


autojump PyCG:
Precision (精准率): 96.620
Recall (召回率): 66.089
F1: 78.490


autojump Depends:
Precision (精准率): 99.213
Recall (召回率): 24.277
F1: 39.009


sqlparse Ae:
Precision (精准率): 86.879
Recall (召回率): 73.795
F1: 79.805


sqlparse PyCG:
Precision (精准率): 95.225
Recall (召回率): 54.066
F1: 68.972


sqlparse Depends:
Precision (精准率): 94.767
Recall (召回率): 24.548
F1: 38.995


sshtunnel Ae:
Precision (精准率): 82.320
Recall (召回率): 86.628
F1: 84.419


sshtunnel PyCG:
Precision (精准率): 100.000
Recall (召回率): 70.930
F1: 82.993


sshtunnel Depends:
Precision (精准率): 100.000
Recall (召回率): 36.628
F1: 53.617


textrank4zh Ae:
Precision (精准率): 88.235
Recall (召回率): 86.207
F1: 87.209


textrank4zh PyCG:
Precision (精准率): 10

asciinema

In [ ]:
name = "asciinema"


pre_clean = []
skip_list = ['test','tests','setup']
replace_list = [
    ('.__init__',''),
    #('multiprocessing.queues.Queue','multiprocessing.Queue'),
    
]


#print(replace_map["asciinema.commands.auth.AuthCommand.__init__"])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=1, Ae_or_pycg=1, all_or_EA=1)
#main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
#main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)

autojump

In [ ]:
name = "autojump"

pre_clean = ['bin']
skip_list = ['test','tests','setup','uninstall','install']
replace_list = [
    ('.__init__',''),
    ('bin.',''),

]

    

#print(replace_map["asciinema.commands.auth.AuthCommand.__init__"])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=1, Ae_or_pycg=1, all_or_EA=1)
#main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
#main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)


fabric

In [ ]:
name = "fabric"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('Socket','socket'),
    ('invoke.config.Config._set','invoke.config.DataProxy._set'),
    ('invoke.Context._set','invoke.config.DataProxy._set'),
    ('fabric.util.debug','util.debug'),
    ('invoke.terminals.pty_size','invoke.pty_size'),
    ('invoke.context.Context._run',"invoke.Context._run"),
    ('invoke.context.Context._sudo',"invoke.Context._sudo"),
    ('invoke.parser.argument.Argument',"invoke.Argument"),
    ('invoke.program.Program.core_args',"invoke.Program.core_args"),
    ('invoke.collection.Collection',"invoke.Collection"),
    ('invoke.program.Program.load_collection',"invoke.Program.load_collection"),
    ('invoke.program.Program.no_tasks_given',"invoke.Program.no_tasks_given"),
    ('invoke.program.Program.update_config',"invoke.Program.update_config"),
    ('invoke.collection.Collection',"invoke.Collection")
]


main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)

face_classification

In [ ]:
name = "face_classification"

pre_clean = ['src']
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('src.',''),

]

#print(replace_map["asciinema.commands.auth.AuthCommand.__init__"])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)


Sublist3r

In [ ]:
name = "Sublist3r"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('re.compile.findall','re.findall'),
    ('requests.sessions.Session.get','requests.Session.get'),
    ('argparse.ArgumentParser.add_argument','argparse.add_argument'),
    ('argparse.ArgumentParser.parse_args','argparse.parse_args'),
    ('threading.Thread.start','threading.start')

]

main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)


In [ ]:
name = "bpytop"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>"),
    #("<builtin>","<builtin>")
]
# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:
name = "furl"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>"),
    #("<builtin>","<builtin>")

]

# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])

main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:
name = "rich_cli"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>"),
    #("<builtin>","<builtin>")
]
# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:
name = "sqlparse"

pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>"),
    #("<builtin>","<builtin>")
]
# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])
main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:
name = "sshtunnel"

pre_clean = []
#skip_list = []
skip_list = ['test','tests','setup','uninstall','install','setup.py','logging']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>")

]

# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])

main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:
name = "textrank4zh"

pre_clean = []
#skip_list = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','<list>'),
    ("<**PyStr**>","<str>"),
    ("<**PyDict**>","<map>"),
    ("<**PySet**>","<set>"),
    ("<**PyTuple**>","<tuple>"),
    ("<**PyNum**>","<num>"),
    ("<**PyBool**>","<bool>")
]

# for r in replace_list:
#     skip_list.append(r[0])
#     skip_list.append(r[1])

main(name, replace_list, skip_list, pre_clean, precision_write=1, recall_write=0, Ae_or_pycg=1, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=0)
main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=0)

In [ ]:


pre_clean = []
skip_list = ['test','tests','setup','uninstall','install','setup.py']
replace_list = [
    ('.__init__',''),
    ('<**PyList**>','list'),
    ("<**PyStr**>","str"),
    ("<**PyDict**>","dict"),
    ("<**PySet**>","set"),
    ("<**PyTuple**>","tuple"),
    ("<**PyNum**>","num"),
    ("<**PyBool**>","bool"),
    ("<builtin>","builtins"),
    ("<**PyFile**>","TextIOWrapper"),
    ("src.",""),
    ('bin.',''),
    # ("mezzanine.","")
]
project_list_id = [
    1,3,4,6,7,8,9,11,12,13,15,16,18,19,20,22,23,24,25,28,29,31,32,33,35,36,37,38,45,46,48,50,52,53,56,57,58
]
precision_all_Ae = 0
recall_all_Ae = 0
f1_all_Ae = 0

precision_all_pycg = 0
recall_all_pycg = 0
f1_all_pycg = 0

precision_all_depends = 0
recall_all_depends = 0
f1_all_depends = 0

for id in project_list_id:
    name = "project{}".format(id)
    replace_list.append((name+".",''))
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=1, all_or_EA=1)
    precision_all_Ae += precision
    recall_all_Ae += recall
    f1_all_Ae += f1
    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=0, all_or_EA=1)
    precision_all_pycg += precision
    recall_all_pycg += recall
    f1_all_pycg += f1

    precision, recall, f1 = main(name, replace_list, skip_list, pre_clean, precision_write=0, recall_write=0, Ae_or_pycg=2, all_or_EA=1)
    precision_all_depends += precision
    recall_all_depends += recall
    f1_all_depends += f1

print("Ae average pre={:.3f}".format(precision_all_Ae/len(project_list_id)))
print("Ae average rec={:.3f}".format(recall_all_Ae/len(project_list_id)))
print("Ae average F1={:.3f}".format(f1_all_Ae/len(project_list_id)))

print("pycg average pre={:.3f}".format(precision_all_pycg/len(project_list_id)))
print("pycg average rec={:.3f}".format(recall_all_pycg/len(project_list_id)))
print("pycg average F1={:.3f}".format(f1_all_pycg/len(project_list_id)))

print("depends average pre={:.3f}".format(precision_all_depends/len(project_list_id)))
print("depends average rec={:.3f}".format(recall_all_depends/len(project_list_id)))
print("depends average F1={:.3f}".format(f1_all_depends/len(project_list_id)))      

